 ## Top Zip Codes for Investment Properties

## Business Understanding


### Project Overview

### Problem Statement

### Objectives

1. To use Time Series Model to determine the top 5 zip codes to invest in
2. To find the top 5 zip codes nationwide with the highest ROI within accepted standard deviation threshold (variability in returns)
3. To Perform Forecasting for Future House Prices 
4. Develop time series models to forecast real estate prices for different zip codes over various time horizons.
5. Evaluate the stability and predictability of real estate prices for different zip codes.
6. Determine potential profit margins based on historical data and forecasted price growth.
7. To Combine insights from the above analyses to make a comprehensive recommendation.
8. To effectively identify cities that are optimal for both short-term and long-term investment
9. To create user-friendly interfaces to facilitate easy access to forecast/ predictions and insights.

## Data Understanding

In this section, we will do the following to get more insights about our dataset before proceeding to subsequent steps.
1. Import the Libraries
2. Load and Explore the Time Series Data
3. Inspect the Data Types
4. Inspect the column values

In [1]:
#Importing the libraries to use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import folium
import matplotlib.dates as mdates
import math
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [2]:
#Loading dataset into pandas DataFrame
df = pd.read_csv('zillow_data.csv')

In [3]:
#preview top 5 rows
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [4]:
#preview last 5 rows
df.tail()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400
14722,95851,89155,Mesquite,NV,Las Vegas,Clark,14723,176400.0,176300.0,176100.0,...,333800,336400,339700,343800,346800,348900,350400,353000,356000,357200


In [5]:
#check shape of the df
df.shape

(14723, 272)

In [6]:
#check general df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [7]:
df.dtypes

RegionID       int64
RegionName     int64
City          object
State         object
Metro         object
               ...  
2017-12        int64
2018-01        int64
2018-02        int64
2018-03        int64
2018-04        int64
Length: 272, dtype: object

### Step 2: Data Preprocessing

In [8]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [9]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [10]:
df.isnull().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
              ... 
2017-12          0
2018-01          0
2018-02          0
2018-03          0
2018-04          0
Length: 272, dtype: int64

In [31]:
df.dtypes

RegionID       int64
RegionName     int64
City          object
State         object
Metro         object
               ...  
2017-12        int64
2018-01        int64
2018-02        int64
2018-03        int64
2018-04        int64
Length: 272, dtype: object

In [11]:
# Checking null values
columns_with_missing_values = df.columns[df.isnull().any()]

# Calculate and print the sum and percentage of missing values for each column
for column in columns_with_missing_values:
    missing_sum = df[column].isnull().sum()
    total_entries = len(df)
    missing_percentage = (missing_sum / total_entries) * 100
    print(f"Column '{column}' has {missing_sum} missing values, which is {missing_percentage:.2f}% of total entries.")

Column 'Metro' has 1043 missing values, which is 7.08% of total entries.
Column '1996-04' has 1039 missing values, which is 7.06% of total entries.
Column '1996-05' has 1039 missing values, which is 7.06% of total entries.
Column '1996-06' has 1039 missing values, which is 7.06% of total entries.
Column '1996-07' has 1039 missing values, which is 7.06% of total entries.
Column '1996-08' has 1039 missing values, which is 7.06% of total entries.
Column '1996-09' has 1039 missing values, which is 7.06% of total entries.
Column '1996-10' has 1039 missing values, which is 7.06% of total entries.
Column '1996-11' has 1039 missing values, which is 7.06% of total entries.
Column '1996-12' has 1039 missing values, which is 7.06% of total entries.
Column '1997-01' has 1039 missing values, which is 7.06% of total entries.
Column '1997-02' has 1039 missing values, which is 7.06% of total entries.
Column '1997-03' has 1039 missing values, which is 7.06% of total entries.
Column '1997-04' has 1039 m

Dropping missing values as they are of lower percentage

In [12]:
#dropping missing values
df1 = df.dropna()

In [13]:
#checking shape of the df after dropping missing values
df1.shape

(12895, 272)

In [14]:
# checking for duplicates
df1.duplicated().value_counts()

False    12895
dtype: int64

In [15]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [16]:
df.values

array([[84654, 60657, 'Chicago', ..., 1030700, 1033800, 1030600],
       [90668, 75070, 'McKinney', ..., 319600, 321100, 321800],
       [91982, 77494, 'Katy', ..., 323000, 326900, 329900],
       ...,
       [75672, 40404, 'Berea', ..., 132500, 133000, 133400],
       [93733, 81225, 'Mount Crested Butte', ..., 705300, 681500, 664400],
       [95851, 89155, 'Mesquite', ..., 353000, 356000, 357200]],
      dtype=object)

In [17]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [18]:
melt_data(df)

,value
time,
1996-04-01,118299.123063
1996-05-01,118419.044139
1996-06-01,118537.423268
1996-07-01,118653.069278
1996-08-01,118780.254312
...,...
2017-12-01,281095.320247
2018-01-01,282657.060382
2018-02-01,284368.688447
